I haven't done this in a while. It used to be Yahoo Finance API, then Quantopian and Quandl, I'm way behind. I hear good things about (AI take the wheel) _Alpha Vantage_.

In [1]:
%pip install joblib

In [2]:
from joblib import Memory

memory = Memory('.cache', verbose=0)

In [10]:
import pandas as pd
import requests
from dotenv import load_dotenv
import os
from functools import lru_cache

load_dotenv()

@memory.cache
def fetch_dailies(ticker):
  url = f"https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&outputsize=full&symbol={ticker}&apikey={os.getenv('ALPHAVANTAGE_API_KEY')}"
  response = requests.get(url)
  response = response.json()
  # Convert the response to a pandas DataFrame
  df = pd.DataFrame(response['Time Series (Daily)']).T
  df.index = pd.to_datetime(df.index)
  df.columns = ['Open', 'High', 'Low', 'Close', 'Volume']
  df['Ticker'] = ticker
  # Convert 'Open' and 'Close' columns to numeric
  df['Open'] = pd.to_numeric(df['Open'], errors='coerce')
  df['Close'] = pd.to_numeric(df['Close'], errors='coerce')

  # Calculate the overnight return
  df['Overnight_Return'] = (df['Open'] / df['Close'].shift(-1)) - 1
  # drop the last row, since we don't have the next day's data
  df = df.iloc[:-1]

  return df


In [11]:
fetch_dailies('AAPL')

________________________________________________________________________________
[Memory] Calling __main__--var-folders-y4-m2rb4mm129b7g3k8hskwsqww0000gn-T-ipykernel-2883560159.fetch_dailies...
fetch_dailies('AAPL')
____________________________________________________fetch_dailies - 1.2s, 0.0min


,Open,High,Low,Close,Volume,Ticker,Overnight_Return
2024-02-29,181.17,182.5700,179.5300,180.75,135487634,AAPL,-0.001378
2024-02-28,182.51,183.1200,180.1300,181.42,48953939,AAPL,-0.000657
2024-02-27,181.10,183.9225,179.5600,182.63,54318851,AAPL,-0.000331
2024-02-26,182.24,182.7600,180.6500,181.16,40867421,AAPL,-0.001534
2024-02-23,185.01,185.0400,182.2300,182.52,45119677,AAPL,0.003471
...,...,...,...,...,...,...,...
1999-11-08,87.75,97.7300,86.7500,96.37,8490400,AAPL,-0.006341
1999-11-05,84.62,88.3700,84.0000,88.31,3721500,AAPL,0.011959
1999-11-04,82.06,85.3700,80.6200,83.62,3384700,AAPL,0.006871
1999-11-03,81.62,83.2500,81.0000,81.50,2932700,AAPL,0.017072


Easy, awesome, I'm game.